# Q2: Community Detection in Random Networks

This notebook demonstrates community detection using different algorithms:
- k-clique communities
- k-clan communities
- k-plex communities
- k-core decomposition

We'll generate a random network with 1000 nodes and compare the characteristics of communities found by each method.

## 1. Import Required Libraries

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
from collections import Counter
import warnings
warnings.filterwarnings('ignore')

# Set random seed for reproducibility
np.random.seed(42)

## 2. Generate Random Network with 1000 Nodes

In [ ]:
# Generate a random network using Erdos-Renyi model
# Using probability p=0.005 to create a moderately connected network
n_nodes = 1000
p = 0.005  # Edge probability

G = nx.erdos_renyi_graph(n_nodes, p, seed=42)

print(f"Network created with {G.number_of_nodes()} nodes and {G.number_of_edges()} edges")
print(f"Average degree: {2 * G.number_of_edges() / G.number_of_nodes():.2f}")
print(f"Density: {nx.density(G):.4f}")
print(f"Number of connected components: {nx.number_connected_components(G)}")

## 3. k-Clique Communities

k-clique communities are defined as the union of all cliques of size k that can be reached through adjacent cliques.

In [ ]:
# Find k-clique communities (using k=3 for triangles)
k_clique = 3
clique_communities = list(nx.community.k_clique_communities(G, k_clique))

print(f"\n=== k-CLIQUE COMMUNITIES (k={k_clique}) ===")
print(f"Number of communities: {len(clique_communities)}")

# Calculate sizes of communities
clique_sizes = [len(comm) for comm in clique_communities]
if clique_sizes:
    print(f"Average community size: {np.mean(clique_sizes):.2f}")
    print(f"Largest community size: {max(clique_sizes)}")
    print(f"Smallest community size: {min(clique_sizes)}")
    print(f"Median community size: {np.median(clique_sizes):.2f}")
else:
    print("No k-clique communities found")

## 4. k-Core Decomposition

k-core is a maximal subgraph where every node has at least k neighbors within the subgraph.

In [ ]:
# Find k-core decomposition
core_numbers = nx.core_number(G)

print(f"\n=== k-CORE DECOMPOSITION ===")
print(f"Maximum core number: {max(core_numbers.values())}")
print(f"Minimum core number: {min(core_numbers.values())}")

# Group nodes by their core number
core_distribution = Counter(core_numbers.values())
print(f"\nCore distribution:")
for k in sorted(core_distribution.keys()):
    print(f"  k-core {k}: {core_distribution[k]} nodes")

# Extract different k-cores
max_k = max(core_numbers.values())
k_cores = {}
for k in range(1, max_k + 1):
    k_cores[k] = nx.k_core(G, k)
    print(f"\nk-core (k={k}): {k_cores[k].number_of_nodes()} nodes, {k_cores[k].number_of_edges()} edges")

## 5. k-Plex Communities

A k-plex is a relaxed clique where each node can miss connections to at most k-1 other nodes in the group.

Note: NetworkX doesn't have a built-in k-plex algorithm, so we'll implement a simple version for finding maximal k-plexes.

In [ ]:
def find_k_plex(G, k, min_size=3):
    """
    Find k-plex subgraphs in G.
    A k-plex is a subgraph where each node is connected to at least n-k nodes in the subgraph,
    where n is the size of the subgraph.
    """
    k_plexes = []
    
    # Use cliques as starting points and relax them
    cliques = list(nx.find_cliques(G))
    
    for clique in cliques:
        if len(clique) >= min_size:
            # Check if it's a k-plex
            is_k_plex = True
            for node in clique:
                neighbors_in_clique = len([n for n in clique if n in G.neighbors(node)])
                # Each node should be connected to at least (size - k) nodes
                if neighbors_in_clique < len(clique) - k:
                    is_k_plex = False
                    break
            
            if is_k_plex:
                k_plexes.append(set(clique))
    
    # Remove duplicates
    unique_plexes = []
    for plex in k_plexes:
        if plex not in unique_plexes:
            unique_plexes.append(plex)
    
    return unique_plexes

# Find k-plex communities
k_plex = 2
plex_communities = find_k_plex(G, k_plex, min_size=3)

print(f"\n=== k-PLEX COMMUNITIES (k={k_plex}) ===")
print(f"Number of k-plex communities found: {len(plex_communities)}")

if plex_communities:
    plex_sizes = [len(comm) for comm in plex_communities]
    print(f"Average k-plex size: {np.mean(plex_sizes):.2f}")
    print(f"Largest k-plex size: {max(plex_sizes)}")
    print(f"Smallest k-plex size: {min(plex_sizes)}")
    print(f"Median k-plex size: {np.median(plex_sizes):.2f}")
else:
    print("No k-plex communities found")

## 6. k-Clan Communities

A k-clan is a k-clique community with the additional constraint that the diameter within the community is at most k.

Note: NetworkX doesn't have a built-in k-clan algorithm, so we'll implement a version based on cliques.

In [ ]:
def find_k_clans(G, k, min_size=3):
    """
    Find k-clan communities.
    A k-clan is a k-clique where the diameter is at most k.
    """
    k_clans = []
    
    # Find all cliques of size at least k
    cliques = [c for c in nx.find_cliques(G) if len(c) >= min_size]
    
    for clique in cliques:
        # Check if the diameter of the subgraph is at most k
        subG = G.subgraph(clique)
        
        if nx.is_connected(subG):
            diameter = nx.diameter(subG)
            if diameter <= k:
                k_clans.append(set(clique))
    
    # Remove duplicates
    unique_clans = []
    for clan in k_clans:
        if clan not in unique_clans:
            unique_clans.append(clan)
    
    return unique_clans

# Find k-clan communities
k_clan = 3
clan_communities = find_k_clans(G, k_clan, min_size=3)

print(f"\n=== k-CLAN COMMUNITIES (k={k_clan}) ===")
print(f"Number of k-clan communities found: {len(clan_communities)}")

if clan_communities:
    clan_sizes = [len(comm) for comm in clan_communities]
    print(f"Average k-clan size: {np.mean(clan_sizes):.2f}")
    print(f"Largest k-clan size: {max(clan_sizes)}")
    print(f"Smallest k-clan size: {min(clan_sizes)}")
    print(f"Median k-clan size: {np.median(clan_sizes):.2f}")
else:
    print("No k-clan communities found")

## 7. Visualization of Communities

In [ ]:
# Create visualizations for each method
fig, axes = plt.subplots(2, 2, figsize=(16, 16))
fig.suptitle('Community Detection Methods Comparison', fontsize=16, fontweight='bold')

# Use the largest connected component for better visualization
largest_cc = max(nx.connected_components(G), key=len)
G_vis = G.subgraph(largest_cc).copy()
pos = nx.spring_layout(G_vis, k=0.5, iterations=50, seed=42)

# 1. k-Clique Communities Visualization
ax1 = axes[0, 0]
nx.draw_networkx_edges(G_vis, pos, alpha=0.1, ax=ax1)
colors_clique = ['lightgray'] * len(G_vis.nodes())
node_list = list(G_vis.nodes())

if clique_communities:
    color_map = plt.cm.get_cmap('tab20', len(clique_communities))
    for i, comm in enumerate(clique_communities[:20]):  # Limit to 20 for visualization
        comm_in_vis = [n for n in comm if n in G_vis.nodes()]
        for node in comm_in_vis:
            idx = node_list.index(node)
            colors_clique[idx] = color_map(i)

nx.draw_networkx_nodes(G_vis, pos, node_color=colors_clique, node_size=30, ax=ax1)
ax1.set_title(f'k-Clique Communities (k={k_clique})\n{len(clique_communities)} communities found', fontsize=12)
ax1.axis('off')

# 2. k-Core Visualization
ax2 = axes[0, 1]
nx.draw_networkx_edges(G_vis, pos, alpha=0.1, ax=ax2)
core_colors = [core_numbers.get(node, 0) for node in G_vis.nodes()]
nx.draw_networkx_nodes(G_vis, pos, node_color=core_colors, node_size=30, 
                       cmap='viridis', ax=ax2, vmin=0, vmax=max(core_numbers.values()))
ax2.set_title(f'k-Core Decomposition\nMax core: {max(core_numbers.values())}', fontsize=12)
ax2.axis('off')

# 3. k-Plex Communities Visualization
ax3 = axes[1, 0]
nx.draw_networkx_edges(G_vis, pos, alpha=0.1, ax=ax3)
colors_plex = ['lightgray'] * len(G_vis.nodes())

if plex_communities:
    color_map = plt.cm.get_cmap('tab20', len(plex_communities))
    for i, comm in enumerate(plex_communities[:20]):  # Limit to 20 for visualization
        comm_in_vis = [n for n in comm if n in G_vis.nodes()]
        for node in comm_in_vis:
            idx = node_list.index(node)
            colors_plex[idx] = color_map(i)

nx.draw_networkx_nodes(G_vis, pos, node_color=colors_plex, node_size=30, ax=ax3)
ax3.set_title(f'k-Plex Communities (k={k_plex})\n{len(plex_communities)} communities found', fontsize=12)
ax3.axis('off')

# 4. k-Clan Communities Visualization
ax4 = axes[1, 1]
nx.draw_networkx_edges(G_vis, pos, alpha=0.1, ax=ax4)
colors_clan = ['lightgray'] * len(G_vis.nodes())

if clan_communities:
    color_map = plt.cm.get_cmap('tab20', len(clan_communities))
    for i, comm in enumerate(clan_communities[:20]):  # Limit to 20 for visualization
        comm_in_vis = [n for n in comm if n in G_vis.nodes()]
        for node in comm_in_vis:
            idx = node_list.index(node)
            colors_clan[idx] = color_map(i)

nx.draw_networkx_nodes(G_vis, pos, node_color=colors_clan, node_size=30, ax=ax4)
ax4.set_title(f'k-Clan Communities (k={k_clan})\n{len(clan_communities)} communities found', fontsize=12)
ax4.axis('off')

plt.tight_layout()
plt.savefig('community_detection_comparison.png', dpi=300, bbox_inches='tight')
plt.show()

print("\nVisualization saved as 'community_detection_comparison.png'")

## 8. Comparison of Community Characteristics

In [ ]:
# Create comparison table
import pandas as pd

comparison_data = {
    'Method': [],
    'Number of Communities': [],
    'Avg Size': [],
    'Min Size': [],
    'Max Size': [],
    'Median Size': [],
    'Total Nodes Covered': []
}

# k-Clique
comparison_data['Method'].append(f'k-Clique (k={k_clique})')
comparison_data['Number of Communities'].append(len(clique_communities))
if clique_sizes:
    comparison_data['Avg Size'].append(f"{np.mean(clique_sizes):.2f}")
    comparison_data['Min Size'].append(min(clique_sizes))
    comparison_data['Max Size'].append(max(clique_sizes))
    comparison_data['Median Size'].append(f"{np.median(clique_sizes):.2f}")
    comparison_data['Total Nodes Covered'].append(len(set().union(*clique_communities)))
else:
    comparison_data['Avg Size'].append('N/A')
    comparison_data['Min Size'].append('N/A')
    comparison_data['Max Size'].append('N/A')
    comparison_data['Median Size'].append('N/A')
    comparison_data['Total Nodes Covered'].append(0)

# k-Core (using cores with at least 1 node)
k_core_communities = [set([n for n, c in core_numbers.items() if c == k]) for k in set(core_numbers.values())]
k_core_communities = [c for c in k_core_communities if len(c) > 0]
k_core_sizes = [len(c) for c in k_core_communities]

comparison_data['Method'].append('k-Core')
comparison_data['Number of Communities'].append(len(k_core_communities))
comparison_data['Avg Size'].append(f"{np.mean(k_core_sizes):.2f}")
comparison_data['Min Size'].append(min(k_core_sizes))
comparison_data['Max Size'].append(max(k_core_sizes))
comparison_data['Median Size'].append(f"{np.median(k_core_sizes):.2f}")
comparison_data['Total Nodes Covered'].append(len(set().union(*k_core_communities)))

# k-Plex
comparison_data['Method'].append(f'k-Plex (k={k_plex})')
comparison_data['Number of Communities'].append(len(plex_communities))
if plex_sizes:
    comparison_data['Avg Size'].append(f"{np.mean(plex_sizes):.2f}")
    comparison_data['Min Size'].append(min(plex_sizes))
    comparison_data['Max Size'].append(max(plex_sizes))
    comparison_data['Median Size'].append(f"{np.median(plex_sizes):.2f}")
    comparison_data['Total Nodes Covered'].append(len(set().union(*plex_communities)))
else:
    comparison_data['Avg Size'].append('N/A')
    comparison_data['Min Size'].append('N/A')
    comparison_data['Max Size'].append('N/A')
    comparison_data['Median Size'].append('N/A')
    comparison_data['Total Nodes Covered'].append(0)

# k-Clan
comparison_data['Method'].append(f'k-Clan (k={k_clan})')
comparison_data['Number of Communities'].append(len(clan_communities))
if clan_sizes:
    comparison_data['Avg Size'].append(f"{np.mean(clan_sizes):.2f}")
    comparison_data['Min Size'].append(min(clan_sizes))
    comparison_data['Max Size'].append(max(clan_sizes))
    comparison_data['Median Size'].append(f"{np.median(clan_sizes):.2f}")
    comparison_data['Total Nodes Covered'].append(len(set().union(*clan_communities)))
else:
    comparison_data['Avg Size'].append('N/A')
    comparison_data['Min Size'].append('N/A')
    comparison_data['Max Size'].append('N/A')
    comparison_data['Median Size'].append('N/A')
    comparison_data['Total Nodes Covered'].append(0)

df_comparison = pd.DataFrame(comparison_data)
print("\n" + "="*80)
print("COMPREHENSIVE COMPARISON OF COMMUNITY DETECTION METHODS")
print("="*80)
print(df_comparison.to_string(index=False))
print("="*80)

## 9. Size Distribution Comparison

In [ ]:
# Plot size distributions
fig, axes = plt.subplots(2, 2, figsize=(14, 10))
fig.suptitle('Community Size Distributions', fontsize=16, fontweight='bold')

# k-Clique
if clique_sizes:
    axes[0, 0].hist(clique_sizes, bins=20, color='skyblue', edgecolor='black', alpha=0.7)
    axes[0, 0].set_xlabel('Community Size')
    axes[0, 0].set_ylabel('Frequency')
    axes[0, 0].set_title(f'k-Clique (k={k_clique})')
    axes[0, 0].grid(True, alpha=0.3)
else:
    axes[0, 0].text(0.5, 0.5, 'No communities found', ha='center', va='center')
    axes[0, 0].set_title(f'k-Clique (k={k_clique})')

# k-Core
axes[0, 1].hist(k_core_sizes, bins=20, color='lightcoral', edgecolor='black', alpha=0.7)
axes[0, 1].set_xlabel('Community Size')
axes[0, 1].set_ylabel('Frequency')
axes[0, 1].set_title('k-Core')
axes[0, 1].grid(True, alpha=0.3)

# k-Plex
if plex_sizes:
    axes[1, 0].hist(plex_sizes, bins=20, color='lightgreen', edgecolor='black', alpha=0.7)
    axes[1, 0].set_xlabel('Community Size')
    axes[1, 0].set_ylabel('Frequency')
    axes[1, 0].set_title(f'k-Plex (k={k_plex})')
    axes[1, 0].grid(True, alpha=0.3)
else:
    axes[1, 0].text(0.5, 0.5, 'No communities found', ha='center', va='center')
    axes[1, 0].set_title(f'k-Plex (k={k_plex})')

# k-Clan
if clan_sizes:
    axes[1, 1].hist(clan_sizes, bins=20, color='plum', edgecolor='black', alpha=0.7)
    axes[1, 1].set_xlabel('Community Size')
    axes[1, 1].set_ylabel('Frequency')
    axes[1, 1].set_title(f'k-Clan (k={k_clan})')
    axes[1, 1].grid(True, alpha=0.3)
else:
    axes[1, 1].text(0.5, 0.5, 'No communities found', ha='center', va='center')
    axes[1, 1].set_title(f'k-Clan (k={k_clan})')

plt.tight_layout()
plt.savefig('community_size_distributions.png', dpi=300, bbox_inches='tight')
plt.show()

print("\nSize distribution visualization saved as 'community_size_distributions.png'")

## 10. Summary and Insights

In [ ]:
print("\n" + "="*80)
print("SUMMARY AND INSIGHTS")
print("="*80)

print("\n1. k-Clique Communities:")
print("   - Identifies communities based on overlapping cliques")
print("   - Tends to find larger, more interconnected communities")
print("   - Communities can overlap (nodes can belong to multiple communities)")

print("\n2. k-Core Decomposition:")
print("   - Identifies hierarchical layers of network density")
print("   - Each node is assigned a core number based on degree")
print("   - Higher k-cores represent more tightly connected groups")
print("   - Non-overlapping partition of the network")

print("\n3. k-Plex Communities:")
print("   - Relaxed version of cliques (allows some missing edges)")
print("   - More flexible than strict cliques")
print("   - Can identify cohesive subgroups with minor gaps in connectivity")

print("\n4. k-Clan Communities:")
print("   - Combines clique structure with diameter constraint")
print("   - Ensures members are within k steps of each other")
print("   - Identifies tightly-knit communities with short paths")

print("\n" + "="*80)
print("KEY DIFFERENCES:")
print("="*80)
print("- k-Clique and k-Clan focus on clique-based structures")
print("- k-Plex allows for relaxed connectivity within communities")
print("- k-Core emphasizes degree-based hierarchy")
print("- k-Clique, k-Plex, and k-Clan can produce overlapping communities")
print("- k-Core produces a strict hierarchical decomposition")
print("="*80)